### KRX 데이터정보시스템을 이용하여 kospi, kosdaq자료 취득
- yahoo의 경우, 데이터 누락이 가끔 발생하여 의도치 않게 분석 왜곡현상이 발생할 수 있음.

In [1]:
from selenium import webdriver as wd
from selenium.webdriver import ActionChains # scroll down 사용하기 위하여 선서
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup as bs

import datetime, time
from datetime import date

import pandas as pd
import numpy as np
import requests
import time
import os, sys

In [2]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
# !pip install chromedriver_autoinstaller

import chromedriver_autoinstaller

driver_name = chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists
                                      # and if it doesn't exist, download it automatically,
                                      # then add chromedriver to path
time.sleep(1)

In [4]:
# driver = wd.Chrome(driver_name)
driver = wd.Chrome(service=Service(ChromeDriverManager().install()))

In [5]:
def get_data_kos(kos_id):

    com_ticker = kos_id
    # 회사이름 입력 Q 버튼
    print("***", com_ticker)
   
    css_id = '#jsMdiMenu > div.lnb_tree > ul > li:nth-child(1) > ul > li.sel > div > \
    div.lnb_tree_box > ul > li.sel.on > ul > li.has5dep.sel.on > ul > li.CI-MDI-MENU-NO-CHILD.sel > a'
    driver.find_element(By.CSS_SELECTOR, css_id).click()
    time.sleep(2)

    # pop up된 입력창에서 회사이름 입력
    driver.find_element(By.ID, 'tboxidxCd_finder_equidx0_0').clear()
    time.sleep(1)

    driver.find_element(By.ID, 'tboxidxCd_finder_equidx0_0').send_keys(com_ticker)
    time.sleep(2)

    # 검색 버튼 푸시
    driver.find_element(By.CSS_SELECTOR, '#btnidxCd_finder_equidx0_0').click()
    time.sleep(2)

    # 테이블에서 최종 선택
    css_sel = '#jsGrid__finder_equidx0_0 > tbody > tr:nth-child(1) > td.tal.pl20'
    #jsGrid__finder_stkisu0_0 > tbody > tr:nth-child(1) > td:nth-child(3)
    #     driver.find_element_by_css_selector(css_sel).click()

    element = WebDriverWait(driver, 20).until(EC.text_to_be_present_in_element((By.CSS_SELECTOR, css_sel), '코스피'))
    # 위 라인은 pop up 창이 사라질 때까지 기다리게 해 줌
    driver.find_element(By.CSS_SELECTOR, css_sel).click()
    
    driver.find_element(By.ID, 'jsSearchButton').click()
    time.sleep(1)
    
    driver.find_element(By.ID, 'jsOpenView_1').click()
    time.sleep(1)

    return

In [6]:
def set_date(start_date, end_date): # 일정 기간 데이터 취득
    # end_date를 먼저 입력하고 start date 입력. 반대로 하면 start date가 이전날짜로  reset되어짐
    driver.find_element(By.ID, 'endDd').clear()
    driver.find_element(By.ID, 'endDd').send_keys(end_date)
    time.sleep(1)
    driver.find_element(By.ID, 'strtDd').clear()
    driver.find_element(By.ID, 'strtDd').send_keys(start_date)
    time.sleep(1)

In [7]:
def get_data_date(start_date, end_date):
    
#     column_name = ['date', 'close', 'change', 'close_cr', 'open', 'high', 'low', 
#                   'vol', 'vol_amount','total_amount', 'total_counts' ]
    column_name = ['date', 'close', 't1', 'close_cr',  'open', 'high', 'low', 'vol', 't2', 't3']
    # ['일자', '종가', '대비', '등락률', '시가', '고가', '저가', '거래량', 
    #                                 '거래대금', '시가총액', '상장주식수']
 
    start_str = start_date.strftime('%Y-%m-%d')
    ed_str = end_date.strftime('%Y-%m-%d')
    set_date(start_str, ed_str)

    # 테이블  취득 버튼 클릭 (우상귀) 아래 에러가 발생하여 현재 보이는 테이블을 받아 사용하는 것으로 일단 회피함(에러미해결)
#     driver.find_element(By.CSS_SELECTOR, '#jsSearchButton').click()
#     time.sleep(5)

    df = pd.read_html(driver.page_source, 
                          attrs={"class": "CI-GRID-BODY-TABLE"}, flavor=["lxml", "bs4"])[0]
    df.columns = column_name
    df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x, "%Y/%m/%d"))
    df_get = df[['date', 'open', 'high', 'low', 'close', 'close_cr', 'vol']]
    
    return df_get

In [8]:
def non_empty_index_df(df_input, start_date, end_date): # 토,일,공휴일등 거래가 없는 일자도 모두 포함
    date_range_ts = pd.date_range(start=start_date, end=end_date)
    df_input.set_index('date', inplace=True)
    df_out = pd.DataFrame(columns = df_input.columns)
    df_out.insert(0, 'date', date_range_ts)
    df_out.set_index('date', inplace=True)
    df_out.update(df_input)
    df_out.reset_index(inplace=True)
    return df_out

In [9]:
def concat_df(df_o, df):
    df_o = pd.concat([df_o, df], ignore_index=True)
    df_o.drop_duplicates(subset=['date'], keep='last', inplace=True)
#     df_o.drop_duplicates(subset=['date'], keep='first', inplace=True)
    df_o.sort_values(by=[df_o.columns[0]], inplace=True)
    df_o.index = np.arange(0, len(df_o))  # 일련 번호 오름차순으로 재 설정
    return df_o

In [10]:
def make_pickle(df, pkl_name):
    pkl_directory = 'data/common_pkl/'
    try:
        if not os.path.exists(pkl_directory):
            os.makedirs(pkl_directory)
    except OSError:
        print("Error: Failed to create the directory.")

    # 데이터 저장: ../data/spx.pkl
    df.to_pickle(pkl_directory+pkl_name)
    df.to_csv(pkl_directory+pkl_name.replace('pkl','csv'))

In [11]:
def get_ticker_data(ticker, startdate, enddate, col_name):
    ydata = yf.Ticker(ticker)
    rdata = ydata.history(period="2y") # 오늘부터 2년치
    rdata.reset_index('Date', inplace=True)
    rdata['Date'] = rdata['Date'].dt.date  # datetime64 to datetime.date()
    rdata = rdata[(rdata['Date'] <= enddate) & (rdata['Date'] >= startdate)] 
    rdata['temp'] = rdata['Close'].shift(1)
    val_temp = (rdata['Close'] - rdata['temp'])/rdata['temp']*100
    rdata[f'{col_name}_cr'] = val_temp.map("{:.2f}%".format)
    rdata = rdata[['Date', 'Close', 'Open', 'High', 'Low', 'Volume', f'{col_name}_cr']] # 필요한 column만 남김
    rdata.columns = ['date', f'{col_name}', 'open', 'high', 'low', 'volume', f'{col_name}_cr'] # column이름 통일
    rdata.reset_index(drop=True, inplace=True) # index번호를 0부터 재정리
    return rdata

In [12]:
# driver.set_window_position(-10000,0) # hide windows
main_url = 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201010105'
# 개별종목 시세추이 data-menu-id: MDC0201020103
driver.get(main_url)
time.sleep(1)

In [ ]:
# 

In [ ]:
driver.find_element(By.ID, 'jsSearchButton').click()

In [ ]:
driver.find_element(By.CSS_SELECTOR, '#jsSearchButton').click()

In [ ]:
# select 개별지수종합정보 side menu
data_menu_id = '#jsMdiMenu > div.lnb_tree > ul > li:nth-child(1) > ul > li.sel > div > div.lnb_tree_box > ul > li.sel > ul > li.has5dep.sel.on > ul > li.CI-MDI-MENU-NO-CHILD.sel > a'
driver.find_element(By.CSS_SELECTOR, data_menu_id).click()
time.sleep(1)

In [ ]:
# 백만원 단위 표시 선정
css_sel = '#MDCSTAT017_FORM > div.CI-MDI-UNIT-WRAP > div > p:nth-child(2) \
           > select.CI-MDI-UNIT-MONEY > option:nth-child(3)'
# driver.find_element_by_css_selector(css_sel).click()  # by_scc 형식이 없어짐.
driver.find_element(By.CSS_SELECTOR, css_sel).click()
time.sleep(1)

In [13]:
code = {'kospi':'코스피', 'kosdaq':'코스닥'}
code = {'kospi':'코스피'}

In [14]:
pkl_directory = 'data/common_pkl/'

In [15]:
for com, val in code.items():
    com_name = f'{val}'
    pkl_name= f'{com}.pkl'
    try :
        df_o = pd.read_pickle(pkl_directory + pkl_name)
        start_date = df_o['date'].iloc[len(df_o)-1]
    except :
        start_date = datetime.date(2022, 1, 1)   # 데이터 취득 시작 일자     

#     start_date = datetime.date(2023, 5, 11)  # 데이터 취득 오류시 일시 사용
        
    end_date = datetime.date.today()
    get_data_kos(com_name)
    df_get = get_data_date(start_date, end_date)
    df_out = non_empty_index_df(df_get, start_date, end_date)

*** 코스피


In [17]:
df_out = non_empty_index_df(df_get, start_date, end_date)

In [19]:
try :
    df_o = concat_df(df_o, df_out) # append df to original df
except :
    df_o = df_out.copy()

C:\Users\kange\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\sorting.py:417: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  indexer = non_nan_idx[non_nans.argsort(kind=kind)]


In [20]:
df_o

,date,kospi,open,high,low,volume,kospi_cr,close,close_cr,vol
0,2021-11-29,2909.32,2906.15,2930.22,2890.78,844400.0,-0.92,NaN,NaN,NaN
1,2021-11-30,2839.01,2932.71,2942.93,2822.73,983556.0,-2.42,NaN,NaN,NaN
2,2021-12-01,2899.72,2860.12,2905.74,2837.03,563927.0,2.14,NaN,NaN,NaN
3,2021-12-02,2945.27,2874.64,2945.27,2874.64,534462.0,1.57,NaN,NaN,NaN
4,2021-12-03,2968.33,2935.93,2975.44,2927.55,486749.0,0.78,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
364,2023-05-19,2537.79,2530.83,2538.31,2524.89,570097.0,0.89,NaN,NaN,NaN
365,2023-05-22,2557.08,2540.03,2564.09,2533.5,593620.0,0.76,NaN,NaN,NaN
366,2023-05-23,2567.55,2565.8,2577.32,2561.1,576045.0,0.41,NaN,NaN,NaN
367,2023-05-23 00:00:00,NaN,2565.8,2577.32,2561.1,NaN,NaN,2567.55,0.41,576045


In [18]:
df_out

,date,open,high,low,close,close_cr,vol
0,2023-05-23,2565.8,2577.32,2561.1,2567.55,0.41,576045
1,2023-05-24,2557.11,2572.21,2555.04,2567.45,-0.0,1072440


In [ ]:
driver.find_element(By.ID, 'jsSearchButton').click()
time.sleep(1)

In [ ]:
driver.find_element(By.ID, 'jsOpenView_1').click()
time.sleep(1)

In [ ]:
start_str = start_date.strftime('%Y-%m-%d')
ed_str = end_date.strftime('%Y-%m-%d')
set_date(start_str, ed_str)

In [ ]:
driver.find_element(By.CSS_SELECTOR, '#jsSearchButton').send_keys(Keys.ENTER)
#jsSearchButton

In [ ]:
stop_tag = driver.find_element(By.CSS_SELECTOR, '#jsSearchButton')

# jsOpenView_1 element 까지 스크롤
action = ActionChains(driver)
action.move_to_element(stop_tag).perform()

In [ ]:
df = pd.read_html(driver.page_source, attrs={"class": "CI-GRID-BODY-TABLE"}, flavor=["lxml", "bs4"])[0]

In [ ]:
df.head()

In [ ]:
# 테이블  취득 버튼 클릭 (우상귀)
driver.find_element(By.ID, 'jsSearchButton').click()
time.sleep(5)

In [ ]:
driver.find_element(By.ID, 'jsSearchButton').click()

In [ ]:

driver.find_element(By.ID, 'jsSearchButton').click()

In [ ]:
df_o.tail()

In [ ]:
df = pd.read_csv(pkl_directory + 'kospi1.csv', encoding='cp949' )
columns = ['date', 'kospi', 't1', 'kospi_cr',  'open', 'high', 'low', 'volume', 't2', 't3']
df.columns = columns
df['date'] = pd.to_datetime(df['date']).dt.date
df_o = df[['date', 'kospi', 'open', 'high', 'low', 'volume', 'kospi_cr']]
df_o = df_o.sort_values(by='date')
df_o.index = np.arange(0,len(df_o))
make_pickle(df_o, 'kospi.pkl')

In [ ]:
df = pd.read_csv(pkl_directory + 'kosdaq1.csv', encoding='cp949' )
columns = ['date', 'kosdaq', 't1', 'kosdaq_cr',  'open', 'high', 'low', 'volume', 't2', 't3']
df.columns = columns
df['date'] = pd.to_datetime(df['date']).dt.date
df_o = df[['date', 'kosdaq', 'open', 'high', 'low', 'volume', 'kosdaq_cr']]
df_o = df_o.sort_values(by='date')
df_o.index = np.arange(0,len(df_o))
make_pickle(df_o, 'kosdaq.pkl')

In [ ]:
pkl_directory = 'data/common_pkl/'

for com, val in code.items():
    com_name = f'{val}'
    pkl_name= f'{com}.pkl'
    try :
        df_o = pd.read_pickle(pkl_directory + pkl_name)
        start_date = df_o['date'].iloc[len(df_o)-1]
    except :
        start_date = datetime.date(2022, 1, 1)   # 데이터 취득 시작 일자     

#     start_date = datetime.date(2023, 5, 11)  # 데이터 취득 오류시 일시 사용
        
    end_date = datetime.date.today()
    get_data_kos(com_name)
    df_get = get_data(start_date, end_date)
    df_out = non_empty_index_df(df_get, start_date, end_date)
#   NaN to null string. cause NaN does not replace original values in the original datafram    
    try :
        df_o = concat_df(df_o, df_out) # append df to original df
    except :
        df_o = df_out.copy()
    df_o = concat_df(df_o, df_out) # append df to original df
    df_o.replace(np.nan, '', inplace=True)
    df_o.to_pickle(pkl_directory+pkl_name)
    df_o.to_csv(pkl_directory+pkl_name.replace('pkl','csv'))
    modification_time.loc[pkl_name][0] = datetime.datetime.now()
    
    print(com_name, f'{i+1}/{total}', end=', ') # 진행상황 확인용
    
# # df_time = pd.DataFrame.from_dict(modification_time, orient='index', columns=['time'])
# modification_time.to_pickle(pkl_directory+'modification_time_company_his.pkl')
# modification_time.to_csv(pkl_directory+'modification_time_company_his.csv')

### 투자자별 현황
 - 거래 정보와 코딩 구성등 형태는 똑 같으나, empty_datetime index 내용과, column selected 내용이 서로 달라서,
    별개의 루틴으로 구성함.

In [ ]:
def date_set(datei): # 하루 하루 데이터를 받아야 함.
# end_date를 먼저 입력하고 start date 입력. 반대로 하면 start date가 이전날짜로  reset되어짐
#     driver.find_element_by_id('endDd').clear()
#     driver.find_element_by_id('endDd').send_keys(datei)
    driver.find_element(By.ID, 'endDd').clear()
    driver.find_element(By.ID, 'endDd').send_keys(datei)
    time.sleep(1)
#     driver.find_element_by_id('strtDd').clear()
#     driver.find_element_by_id('strtDd').send_keys(datei)
    driver.find_element(By.ID, 'strtDd').clear()
    driver.find_element(By.ID, 'strtDd').send_keys(datei)
    time.sleep(1)
    return

In [ ]:
def push_button_1(): # 조회 button push
    xp = '/html/body/div[2]/section[2]/section/section/div/div[2]/form/div[1]/div/a' 
    # use full xpath to avoid 'Message: element not interactable' Error
#     driver.find_element_by_xpath(xp).click()
    driver.find_element(By.XPATH, xp).click()
    time.sleep(1) # 여유시간 배분
    css_sel = 'div.loading-bar-wrap.small' # 각기 다른 loading 페이지에서 공통적으로 사용됨
    element = WebDriverWait(driver, 20).until(EC.invisibility_of_element_located((By.CSS_SELECTOR, css_sel)))
    # 위 라인은 pop up 창이 사라질 때까지 기다리게 해 줌
    time.sleep(1) # 여유시간 배분
    return

In [ ]:
base_data_directory = './data/base_data/stock_market_holydays/'
opening_days_kor = pd.read_pickle(base_data_directory+'opening_days_kor.pkl') # 한국 개장일 데이터 
def is_opening_day(date): # 개장일 확인
    date = datetime.datetime.strptime(date, "%Y%m%d").date()
    if date in list(opening_days_kor):
        return True
    else:
        return False

In [ ]:
index_name = ['investor', 'sell_quantity', 'buy_quantity', 'pure_buy_quantity', 'sell', 'buy', 'pure_buy']
column_name = ['financial', 'insurance', 'invtrust', 'privequity', 'bank', 'financeetc', 'pension',
              'institution', 'corporateetc', 'retail', 'foreigner', 'foreigneretc', 'total' ]
def get_daily_data(date_range):
    df_org = None
    for datei in date_range: 
        
        if not is_opening_day(datei): # 나중(5월 2일 이후)에 개장일만 수집하도록 수정할 것
            continue

        date_set(datei)
        push_button_1()
        df = pd.read_html(driver.page_source, 
                          attrs={"class": "CI-GRID-BODY-TABLE"}, flavor=["lxml", "bs4"])[0]
        df.columns = index_name
        df_new = df[['investor', 'pure_buy']] # 순매수 금액
        df_new.set_index('investor', inplace=True)
        dft = df_new.T
        dft.columns = column_name
        dft.insert(0, "date", datetime.datetime.strptime(datei, "%Y%m%d"))
        dft.reset_index(drop=True, inplace=True)
        if df_org is None:
            df_org = dft.copy()
            continue
#         df_org = df_org.append(dft, ignore_index=True) # append will be depreciated
        df_org =pd.concat([df_org,dft], ignore_index=True)
        
    return df_org

In [ ]:
def get_data_company_investor(com_name, start_date, end_date):
    
    com_ticker = com_name[:6]
    # 회사이름 입력 Q 버튼
#     driver.find_element_by_css_selector('#btnisuCd_finder_stkisu0_1').click()
    driver.find_element(By.CSS_SELECTOR, '#btnisuCd_finder_stkisu0_1').click()
    time.sleep(2)

    # pop up된 입력창에서 회사이름 입력
#     driver.find_element_by_id('searchText__finder_stkisu0_1').clear()
    driver.find_element(By.ID, 'searchText__finder_stkisu0_1').clear()
    time.sleep(1)
#     driver.find_element_by_id('searchText__finder_stkisu0_1').send_keys(com_name)
    driver.find_element(By.ID, 'searchText__finder_stkisu0_1').send_keys(com_name)
    time.sleep(2)

    # 검색 버튼 푸시
#     driver.find_element_by_css_selector('#searchBtn__finder_stkisu0_1').click()
    driver.find_element(By.CSS_SELECTOR, '#searchBtn__finder_stkisu0_1').click()
    time.sleep(2)

    # 테이블에서 최종 선택
    css_sel = '#jsGrid__finder_stkisu0_1 > tbody > tr:nth-child(1) > td:nth-child(1)'
#     driver.find_element_by_css_selector(css_sel).click()

    element = WebDriverWait(driver, 20).until(EC.text_to_be_present_in_element((By.CSS_SELECTOR, css_sel), com_ticker))
    driver.find_element(By.CSS_SELECTOR, css_sel).click()
    time.sleep(2)
    
    date_range = convert_date(start_date, end_date)
    return get_daily_data(date_range)

In [ ]:
def convert_date(start_date, end_date):
    date_range_ts = pd.date_range(start=start_date, end=end_date)
    date_range = []
    for x in date_range_ts:
        date_range.append(datetime.datetime.strftime(x, "%Y%m%d"))
    return date_range

In [ ]:
# 투자자별 URL로 변경
main_url = 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020203'
driver.get(main_url)
time.sleep(2)

In [ ]:
# Message: element not interactable Error 방지용. 클릭하기 위하여는 그 위치가 클릭할 수 있게 노출되어 있어야 함
# 투자자별 거래실적 버튼이 위치한 곳으로 화면 scroll 

# id가 jsOpenView_1 인 element 를 찾음
# stop_tag = driver.find_element_by_id('jsOpenView_1')
stop_tag = driver.find_element(By.ID, 'jsOpenView_1')

# jsOpenView_1 element 까지 스크롤
action = ActionChains(driver)
action.move_to_element(stop_tag).perform()

In [ ]:
# 투자자별 거래 실적 버튼 클릭
# driver.find_element_by_id('jsOpenView_1').click()
driver.find_element(By.ID, 'jsOpenView_1').click()
time.sleep(2)

In [ ]:
# 백만원 단위 표시 선정
css_sel = '#MDCSTAT023_FORM > div.CI-MDI-UNIT-WRAP > div > p:nth-child(2) > select.CI-MDI-UNIT-MONEY > option:nth-child(3)'
# driver.find_element_by_css_selector(css_sel).click()
driver.find_element(By.CSS_SELECTOR, css_sel).click()
time.sleep(1)

In [ ]:
pkl_directory = 'data/company_pkl/'
# modification_time = {} # 데이터가 생성된 시간 저장 dictionary
modification_time = pd.read_pickle(pkl_directory + 'modification_time_company_inv.pkl')
total = len(code)

for i, (key, val) in enumerate(code.items()):
    com_name = "/".join([key, val[0]])
    pkl_name= '{}_investors.pkl'.format(val[1])
    try :
        df_o = pd.read_pickle(pkl_directory + pkl_name)
        start_date = df_o['date'].iloc[len(df_o)-1]
    except :
        start_date = datetime.date(2022, 1, 1)   # 데이터 취득 시작 일자 
        
#     start_date = datetime.date(2023, 5, 11) # 데이터 취득 오류시 일시 사용

    end_date = datetime.date.today()
    df_out = get_data_company_investor(com_name, start_date, end_date)
    try :
        df_out = df_out[df_o.columns]     
        df_o = concat_df(df_o, df_out) # append df to original df
    except :
        df_col = ['date', 'retail', 'foreigner', 'institution', 'financial', 'invtrust',
                  'pension', 'privequity', 'bank', 'insurance', 'financeetc',
                  'corporateetc', 'foreigneretc']
        df_out = df_out[df_col]
        df_o = df_out.copy()
    df_o.to_pickle(pkl_directory+pkl_name)
    df_o.to_csv(pkl_directory+pkl_name.replace('pkl','csv'))
    modification_time.loc[pkl_name][0] = datetime.datetime.now()
    
    print(com_name, f'{i+1}/{total}', end=', ') # 진행상황 확인용
    
# df_time = pd.DataFrame.from_dict(modification_time, orient='index', columns=['time'])
modification_time.to_pickle(pkl_directory+'modification_time_company_inv.pkl')
modification_time.to_csv(pkl_directory+'modification_time_company_inv.csv')

In [ ]:
driver.close()
driver.quit()